# 层和块
:label:`sec_model_construction`

之前首次介绍神经网络时，我们关注的是具有单一输出的线性模型。
在这里，整个模型只有**一个输出**。
注意，单个神经网络
（1）接受一些输入；
（2）生成相应的标量输出；
（3）具有一组相关 *参数*（parameters），更新这些参数可以优化某目标函数。

然后，当考虑具有多个输出的网络时，
我们利用矢量化算法来描述整层神经元。
像单个神经元一样，层（1）接受一组输入，
（2）生成相应的输出，
（3）由一组可调整参数描述。
当我们使用softmax回归时，一个单层本身就是模型。
然而，即使我们随后引入了多层感知机，我们仍然可以认为该模型保留了上面所说的基本架构。

对于多层感知机而言，整个模型及其组成层都是这种架构。
整个模型接受原始输入（特征），生成输出（预测），
并包含一些参数（所有组成层的参数集合）。
同样，每个单独的层接收输入（由前一层提供），
生成输出（到下一层的输入），并且具有一组可调参数，
这些参数根据从下一层反向传播的信号进行更新。

事实证明，**研究讨论“比单个层大”但“比整个模型小”的组件更有价值**。
例如，在计算机视觉中广泛流行的ResNet-152架构就有数百层，
这些层是由*层组*（groups of layers）的重复模式组成。
这个ResNet架构赢得了2015年ImageNet和COCO计算机视觉比赛
的识别和检测任务 :cite:`He.Zhang.Ren.ea.2016`。
目前ResNet架构仍然是许多视觉任务的首选架构。
在其他的领域，如自然语言处理和语音，
<u>层组以各种重复模式排列</u>的类似架构现在也是普遍存在。

为了实现这些复杂的网络，我们引入了神经网络*块*的概念。
*块*（block）可以描述***单个层、由多个层组成的组件或整个模型本身***。
使用块进行抽象的一个好处是可以将一些块组合成更大的组件，
这一过程通常是递归的，如 :numref:`fig_blocks`所示。
通过定义代码来按需生成任意复杂度的块，
我们可以通过简洁的代码实现复杂的神经网络。

![多个层被组合成块，形成更大的模型](../img/blocks.svg)
:label:`fig_blocks`

从编程的角度来看，块由*类*（class）表示。
它的<u>任何子类都必须定义一个将其输入转换为输出的前向传播函数，
并且必须存储任何必需的参数</u>。
注意，有些块不需要任何参数。
最后，为了计算梯度，块必须具有反向传播函数。
在定义我们自己的块时，由于自动微分（在 :numref:`sec_autograd` 中引入）
提供了一些后端实现，我们只需要考虑前向传播函数和必需的参数。

在构造自定义块之前，(**我们先回顾一下多层感知机**)
（ :numref:`sec_mlp_concise` ）的代码。
下面的代码生成一个网络，其中包含一个具有256个单元和ReLU激活函数的全连接隐藏层，
然后是一个具有10个隐藏单元且不带激活函数的全连接输出层。


In [13]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
# layer 1：nn.Linear(20, 256)
# layer 1-activation ：nn.ReLU()
# layer 2：nn.Linear(256, 10)

X = torch.rand(2, 20)
net(X)

tensor([[-0.1058,  0.0409, -0.0153,  0.3156,  0.3573, -0.1157,  0.0821, -0.0207,
         -0.0098, -0.1917],
        [-0.1992,  0.0129,  0.0347,  0.2656,  0.4532, -0.1473,  0.0455,  0.0471,
          0.0208, -0.3740]], grad_fn=<AddmmBackward0>)

在这个例子中，我们通过实例化`nn.Sequential`来构建我们的模型，
层的执行顺序是作为参数传递的。
简而言之，(**`nn.Sequential`定义了一种特殊的`Module`**)，
即在PyTorch中表示一个块的类，
它维护了一个由`Module`组成的有序列表。
注意，两个全连接层都是`Linear`类的实例，
`Linear`类本身就是`Module`的子类。
另外，到目前为止，我们一直在通过`net(X)`调用我们的模型来获得模型的输出。
这实际上是`net.__call__(X)`的简写。
这个前向传播函数非常简单：
>它将列表中的每个块连接在一起，将每个块的输出作为下一个块的输入。


## [**自定义块**]

要想直观地了解块是如何工作的，最简单的方法就是自己实现一个。
在实现我们自定义块之前，我们简要总结一下每个块必须提供的基本功能。


1. 将输入数据作为其前向传播函数的参数。
1. 通过前向传播函数来生成输出。请注意，输出的形状可能与输入的形状不同。例如，我们上面模型中的第一个全连接的层接收一个20维的输入，但是返回一个维度为256的输出。
1. 计算其输出关于输入的梯度，可通过其反向传播函数进行访问。*通常这是自动发生的*。
1. 存储和访问前向传播计算所需的参数。
1. 根据需要初始化模型参数。


在下面的代码片段中，我们从零开始编写一个块。
它包含一个多层感知机，其具有256个隐藏单元的隐藏层和一个10维输出层。
注意，下面的`MLP`类继承了表示块的类。
我们的实现只需要提供我们自己的构造函数（Python中的`__init__`函数）和前向传播函数。


In [8]:
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)  # 输出层

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))
    

我们首先看一下前向传播函数，它以`X`作为输入，
计算带有激活函数的隐藏表示，并输出其*未规范化*的输出值。
在这个`MLP`实现中，两个层都是实例变量。
要了解这为什么是合理的，可以想象实例化两个多层感知机（`net1`和`net2`），
并根据不同的数据对它们进行训练。
当然，我们希望它们学到两种不同的模型。

接着我们[**实例化多层感知机的层，然后在每次调用前向传播函数时调用这些层**]。
注意一些关键细节：
首先，我们定制的`__init__`函数通过`super().__init__()`
调用父类的`__init__`函数，
省去了重复编写模版代码的痛苦。
然后，我们实例化两个全连接层，
分别为`self.hidden`和`self.out`。
注意，除非我们实现一个新的运算符，
否则我们不必担心反向传播函数或参数初始化，
系统将自动生成这些。

我们来试一下这个函数：


In [14]:
net = MLP() #在 PyTorch 中，当你调用 net(X) 时，PyTorch 会自动调用 net.forward(X)。
# PyTorch 只认 forward 这个名字。其他名字不会被 __call__ 调用。
net(X)

tensor([[-0.0318,  0.1595,  0.0921, -0.0976,  0.2274, -0.2306, -0.1101,  0.0445,
          0.2111, -0.0936],
        [-0.0540,  0.0043,  0.0166, -0.0916,  0.1144, -0.2128, -0.1500,  0.2444,
          0.2816,  0.0712]], grad_fn=<AddmmBackward0>)

块的一个主要优点是它的多功能性。
我们可以子类化块以创建层（如全连接层的类）、
整个模型（如上面的`MLP`类）或具有中等复杂度的各种组件。
我们在接下来的章节中充分利用了这种多功能性，
比如在处理卷积神经网络时。

## [**顺序块**]

现在我们可以更仔细地看看`Sequential`类是如何工作的，
回想一下`Sequential`的设计是为了把其他模块串起来。
为了构建我们自己的简化的`MySequential`，
我们只需要定义两个关键函数：

1. 一种将块逐个追加到列表中的函数；(用于嵌套其他简单的block)
1. 一种前向传播函数，用于将输入按追加块的顺序传递给块组成的“链条”。

下面的`MySequential`类提供了与默认`Sequential`类相同的功能。


In [10]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

`__init__`函数将每个模块逐个添加到有序字典`_modules`中。
读者可能会好奇为什么每个`Module`都有一个`_modules`属性？
以及为什么我们使用它而不是自己定义一个Python列表？
简而言之，`_modules`的主要优点是：
在模块的参数初始化过程中，
系统知道在`_modules`字典中查找需要初始化参数的子块。


当`MySequential`的前向传播函数被调用时，
每个添加的块都按照它们*被添加的顺序*执行。
现在可以使用我们的`MySequential`类重新实现多层感知机。


In [11]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.1988,  0.0211,  0.0829,  0.0165, -0.0109, -0.0705,  0.2683,  0.2187,
         -0.0580,  0.3442],
        [-0.1546,  0.0704,  0.0736, -0.0728, -0.1218,  0.0234,  0.1662,  0.2623,
         -0.0194,  0.3582]], grad_fn=<AddmmBackward0>)

请注意，`MySequential`的用法与之前为`Sequential`类编写的代码相同
（如 :numref:`sec_mlp_concise` 中所述）。

## [**在前向传播函数中执行代码**]

`Sequential`类使模型构造变得简单，
允许我们组合新的架构，而不必定义自己的类。
然而，并不是所有的架构都是简单的顺序架构。
当需要更强的灵活性时，我们需要定义自己的块。
例如，我们可能希望在前向传播函数中执行Python的控制流。
此外，我们可能希望执行任意的数学运算，
而不是简单地依赖预定义的神经网络层。

到目前为止，
我们网络中的所有操作都对网络的激活值及网络的参数起作用。
然而，有时我们可能希望合并既不是上一层的结果也不是可更新参数的项，
我们称之为*常数参数*（constant parameter）。
例如，我们需要一个计算函数
$f(\mathbf{x},\mathbf{w}) = c \cdot \mathbf{w}^\top \mathbf{x}$的层，
其中$\mathbf{x}$是输入，
$\mathbf{w}$是参数，
$c$是某个在优化过程中没有更新的指定常量。
因此我们实现了一个`FixedHiddenMLP`类，如下所示：


In [ ]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False) #常数参数C，且常数参数C不参与梯度更新，因此在计算中保留为常数
        #  nn.Linear(in_features, out_features) 时，PyTorch 会自动为你初始化权重（weight）和偏置（bias）参数，
        # 并将它们注册为可学习的模型参数（nn.Parameter），即参与梯度更新。
        self.linear = nn.Linear(20, 20) 

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

1.`nn.Linear` 的内部实现（简化版）

当你写：
```python
self.linear = nn.Linear(20, 20)
```

PyTorch 内部大致做了以下事情（伪代码）：

```python
class Linear(nn.Module):
    def __init__(self, in_features, out_features, bias=True):
        super().__init__()
        # 自动创建并注册权重参数
        self.weight = nn.Parameter(torch.empty(out_features, in_features))
        if bias:
            self.bias = nn.Parameter(torch.empty(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()  # 调用初始化方法
```

- `nn.Parameter` 是 `torch.Tensor` 的子类，**特殊之处在于：当它被赋值给 `nn.Module` 的属性时，会被自动加入到模块的参数列表中**。
- 这样，当你调用 `model.parameters()` 时，这些 `weight` 和 `bias` 就会被包含进去，从而被优化器（如 SGD、Adam）更新。

---

2. 初始化方式

`reset_parameters()` 默认使用 **Kaiming Uniform（也叫 He Uniform）初始化**（从 PyTorch 1.0+ 开始），具体为：

- 权重 `weight`：从均匀分布  
  $$
  \mathcal{U}\left(-\sqrt{\frac{1}{\text{in\_features}}},\ \sqrt{\frac{1}{\text{in\_features}}} \right)
  $$
  （实际上更精确的是 `kaiming_uniform_`，考虑了 ReLU 的增益）

- 偏置 `bias`：初始化为 0

> 📌 你不需要手动初始化，除非有特殊需求（比如 Xavier、正态分布等）。

3. 如何查看这些参数？

你可以直接访问它们：

```python
net = FixedHiddenMLP()
print(net.linear.weight.shape)   # torch.Size([20, 20])
print(net.linear.bias.shape)     # torch.Size([20])

print(list(net.parameters()))    # 包含 weight 和 bias
```

输出示例：
```python
Parameter containing:
tensor([[ 0.123, -0.456, ...,  0.789],
        ...
       ], requires_grad=True)

Parameter containing:
tensor([0., 0., ..., 0.], requires_grad=True)
```

注意：`requires_grad=True` 表示这些参数会参与梯度计算和更新。

4. 对比：你定义的 `rand_weight`

```python
self.rand_weight = torch.rand((20, 20), requires_grad=False)
```

- 这只是一个普通张量（`torch.Tensor`），**不是 `nn.Parameter`**
- 即使你设 `requires_grad=True`，它也不会被 `model.parameters()` 返回（除非你用 `register_buffer` 或转为 `Parameter`）
- 所以它**不会被优化器更新**

而 `nn.Linear` 的 `weight` 和 `bias` 是 `nn.Parameter`，所以会被自动管理。

在这个`FixedHiddenMLP`模型中，我们实现了一个隐藏层，
其权重（`self.rand_weight`）在实例化时被随机初始化，之后为常量。
这个权重不是一个模型参数，因此它永远不会被反向传播更新。
然后，神经网络将这个固定层的输出通过一个全连接层。

注意，在返回输出之前，模型做了一些不寻常的事情：
它运行了一个while循环，在$L_1$范数大于$1$的条件下，
将输出向量除以$2$，直到它满足条件为止。
最后，模型返回了`X`中所有项的和。
注意，此操作可能不会常用于在任何实际任务中，
我们只展示如何将任意代码集成到神经网络计算的流程中。


In [18]:
net = FixedHiddenMLP()
# print(net.linear.weight.data) 
net(X)

tensor(-0.0588, grad_fn=<SumBackward0>)

我们可以[**混合搭配各种组合块的方法**]。
在下面的例子中，我们以一些想到的方法嵌套块。


In [19]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.0511, grad_fn=<SumBackward0>)

## 效率


读者可能会开始担心操作效率的问题。
毕竟，我们在一个高性能的深度学习库中进行了大量的字典查找、
代码执行和许多其他的Python代码。
Python的问题[全局解释器锁](https://wiki.python.org/moin/GlobalInterpreterLock)
是众所周知的。
在深度学习环境中，我们担心速度极快的GPU可能要等到CPU运行Python代码后才能运行另一个作业。


## 小结

* 一个块可以由许多层组成；一个块可以由许多块组成。
* 块可以包含代码。
* 块负责大量的内部处理，包括参数初始化和反向传播。
* 层和块的顺序连接由`Sequential`块处理。

## 练习

1. 如果将`MySequential`中存储块的方式更改为Python列表，会出现什么样的问题？
1. 实现一个块，它以两个块为参数，例如`net1`和`net2`，并返回前向传播中两个网络的串联输出。这也被称为平行块。
1. 假设我们想要连接同一网络的多个实例。实现一个函数，该函数生成同一个块的多个实例，并在此基础上构建更大的网络。


[Discussions](https://discuss.d2l.ai/t/1827)


1. 实现一个块，它以两个块为参数，例如`net1`和`net2`，并返回前向传播中两个网络的串联输出。这也被称为平行块。

---

✅ 实现一个**并行块（Parallel Block）**

> **要求**：接收两个网络 `net1` 和 `net2`，在前向传播中**并行**通过两个网络，然后**拼接（concatenate）输出**。

注意：题目说“串联输出”，但从上下文“parallel block”来看，应为**并行结构**（parallel），即两个网络同时处理输入，再合并结果。  
如果是“串联（sequential）”，那就是 `net2(net1(x))`，但那就不是 parallel 了。

所以这里我们实现 **并行 + 拼接**。



In [20]:

import torch
import torch.nn as nn

class ParallelBlock(nn.Module):
    def __init__(self, net1, net2):
        super().__init__()
        self.net1 = net1
        self.net2 = net2

    def forward(self, x):
        out1 = self.net1(x)
        out2 = self.net2(x)
        # 沿最后一个维度拼接（通常是特征维度）
        return torch.cat([out1, out2], dim=-1)


# 使用示例：
# 定义两个子网络
net1 = nn.Sequential(nn.Linear(10, 5), nn.ReLU())
net2 = nn.Sequential(nn.Linear(10, 3), nn.Tanh())

# 构建并行块
parallel_net = ParallelBlock(net1, net2)

# 输入
x = torch.randn(2, 10)
output = parallel_net(x)  # shape: (2, 5+3) = (2, 8)
print(output.shape)  # torch.Size([2, 8])


torch.Size([2, 8])



假设我们想要连接同一网络的多个实例。实现一个函数，该函数生成同一个块的多个实例，并在此基础上构建更大的网络。

---

✅ 任务 ：实现一个函数，生成同一块的多个实例，并构建更大网络

> **要求**：给定一个“块模板”（如 `nn.Linear` 或自定义模块），复制 `n` 份，组成新网络。

常见用途：
- 多头注意力（多个相同的投影层）
- 集成学习（多个相同结构的子网络）
- 特征提取的多尺度分支


In [21]:

# 实现方式（推荐使用 `nn.ModuleList`）

def replicate_block(block_fn, n_replicas):
    """
    创建 n_replicas 个 block_fn() 的独立实例，并放入 ModuleList
    
    Args:
        block_fn: 一个可调用对象，返回 nn.Module 实例（如 lambda: nn.Linear(10, 5)）
        n_replicas: 复制次数
    
    Returns:
        nn.ModuleList 包含 n_replicas 个独立模块
    """
    blocks = []
    for _ in range(n_replicas):
        blocks.append(block_fn())
    return nn.ModuleList(blocks)




> ⚠️ 注意：必须每次调用 `block_fn()` 创建**新实例**，否则所有“副本”会共享同一个对象！

#### 构建更大网络的示例

In [23]:

# 示例 1：并行多头（拼接输出）

class MultiHeadParallel(nn.Module):
    def __init__(self, block_fn, n_heads):
        super().__init__()
        self.heads = replicate_block(block_fn, n_heads)
    
    def forward(self, x):
        outputs = [head(x) for head in self.heads]
        return torch.cat(outputs, dim=-1)

# 使用
head_fn = lambda: nn.Sequential(nn.Linear(10, 4), nn.ReLU())
multi_head = MultiHeadParallel(head_fn, n_heads=3)
x = torch.randn(2, 10)
out = multi_head(x)  # shape: (2, 12)
print(out.shape)

torch.Size([2, 12])


In [25]:
# 示例 2：串行堆叠（Sequential）

def stack_blocks(block_fn, n_layers):
    """将多个相同结构的块串行堆叠"""
    layers = []
    for _ in range(n_layers):
        layers.append(block_fn())
    return nn.Sequential(*layers)

# 使用：3 层 MLP
layer_fn = lambda: nn.Sequential(nn.Linear(64, 64), nn.ReLU())
deep_net = stack_blocks(layer_fn, 3)
print(deep_net)

Sequential(
  (0): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): ReLU()
  )
  (1): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): ReLU()
  )
  (2): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): ReLU()
  )
)



🔍 关键注意事项

❌ 错误做法：共享同一个实例

```python
# 危险！所有“副本”其实是同一个对象
bad_blocks = [my_block] * n   # ❌ 参数完全共享，不是独立副本
```

✅ 正确做法：每次都新建

```python
good_blocks = [block_fn() for _ in range(n)]  # ✅ 独立参数
```

 为什么用 `nn.ModuleList` 而不是普通 list？

- `nn.ModuleList` 是 PyTorch 提供的容器，能**自动注册子模块**
- 如果你用普通 Python list 存 `nn.Module`，PyTorch **无法追踪其参数**！
  ```python
  self.layers = [nn.Linear(10,5), nn.Linear(10,5)]  # ❌ 参数不会被 model.parameters() 返回！
  self.layers = nn.ModuleList([nn.Linear(10,5), nn.Linear(10,5)])  # ✅ 正确
  ```
---

✅ 总结

| 任务 | 实现要点 |
|------|--------|
| **并行块** | 两个网络同时前向，用 `torch.cat` 拼接输出 |
| **复制块** | 用 `nn.ModuleList` + 循环调用 `block_fn()` 创建独立实例 |
| **关键原则** | 每个副本必须是**新对象**，且用 PyTorch 容器（`ModuleList`/`Sequential`）管理 |

这样就能灵活构建各种复杂网络结构！